In [1]:
exception Unimplemented

exception Unimplemented


In [2]:
exception UnexpectedError

exception UnexpectedError


In [3]:
type var = string

type var = string


In [4]:
let fstring = Printf.sprintf

val fstring : ('a, unit, string) format -> 'a = <fun>


In [5]:
type typ = 
    | TUnit
    | Loli   of typ * typ
    | Tensor of typ * typ
    | Sum    of typ * typ

type typ = TUnit | Loli of typ * typ | Tensor of typ * typ | Sum of typ * typ


In [6]:
let rec printtype t = 
    match t with
    | TUnit           -> "1"
    | Loli (t1, t2)   -> fstring "(%s⊸%s)" (printtype t1) (printtype t2)
    | Tensor (t1, t2) -> fstring "(%s⊗%s)" (printtype t1) (printtype t2)
    | Sum (t1, t2)    -> fstring "(%s+%s)" (printtype t1) (printtype t2);;
    
printtype (Loli(Tensor(TUnit, TUnit), TUnit))

val printtype : typ -> string = <fun>


- : string = "((1⊗1)⊸1)"


In [7]:
type expr = 
    | EUnit
    | Var    of var
    | Lambda of var * expr
    | App    of expr * expr
    | Pair   of expr * expr
    | Unpair of var * var * expr * expr
    | Annot  of expr * typ
    | L      of expr
    | R      of expr
    | Case   of expr * var * expr * var * expr

type expr =
    EUnit
  | Var of var
  | Lambda of var * expr
  | App of expr * expr
  | Pair of expr * expr
  | Unpair of var * var * expr * expr
  | Annot of expr * typ
  | L of expr
  | R of expr
  | Case of expr * var * expr * var * expr


In [8]:
let rec printexpr e =
    match e with
    | EUnit -> "()"
    | Var x -> x
    | Lambda (x, e')            -> fstring "λ%s.%s" x (printexpr e')
    | App (e1, e2)              -> fstring "(%s)(%s)" (printexpr e1) (printexpr e2)
    | Pair (e1, e2)             -> fstring "(%s, %s)" (printexpr e1) (printexpr e2)
    | Unpair (x1, x2, e1, e2)   -> fstring "let (%s,%s) = %s in %s" x1 x2 (printexpr e1) (printexpr e2)
    | Annot (e', t)             -> fstring "(%s: %s)" (printexpr e') (printtype t)
    | L e'                      -> fstring "L(%s)" (printexpr e')
    | R e'                      -> fstring "R(%s)" (printexpr e')
    | Case (e', x1, e1, x2, e2) -> fstring "case(%s, L(%s)->%s, R(%s)->%s)" (printexpr e') x1 (printexpr e1) x2 (printexpr e2);;
    
printexpr (Unpair("x1", "x2", Pair(Lambda("x", Var "x"), Var "y"), App(Var "x1", Var "x2")))

val printexpr : expr -> var = <fun>


- : var = "let (x1,x2) = (λx.x, y) in (x1)(x2)"


In [9]:
type usage = Used | Fresh
type state = {var: var; used: usage; typ: typ}

let mkstate v u t = {var=v; used=u; typ=t}
let fresh v t = mkstate v Fresh t

type ctx = state list

type usage = Used | Fresh


type state = { var : var; used : usage; typ : typ; }


val mkstate : var -> usage -> typ -> state = <fun>


val fresh : var -> typ -> state = <fun>


type ctx = state list


In [10]:
let printusage u = match u with Fresh->"1"| Used->"0"

val printusage : usage -> string = <fun>


In [11]:
let printstate s = fstring "%s^%s: %s" s.var (printusage s.used) (printtype s.typ) 

val printstate : state -> string = <fun>


In [12]:
let printctx c =
    let rec loop c = 
        match c with
        | []    -> ""
        | [x]     -> printstate x
        | x::xs -> fstring "%s; %s" (printstate x) (loop xs)
    in fstring "Γ = [%s]" (loop c)

val printctx : state list -> string = <fun>


In [13]:
print_endline (printctx [fresh "x" TUnit; fresh "y" (Loli(TUnit, TUnit))]);;

Γ = [x^1: 1; y^1: (1⊸1)]


- : unit = ()


In [14]:
type errorinfo = string

type 'a result = Value of 'a | Error of errorinfo

type errorinfo = string


type 'a result = Value of 'a | Error of errorinfo


In [15]:
type 'a t = ctx -> ('a * ctx) result

let return (x: 'a) : 'a t = fun (state: ctx) -> Value(x, state)

let error (x: errorinfo) : 'a t = fun (_: ctx) -> Error x

let bind (x: 'a t) (f: 'a -> 'b t) : 'b t = fun (state: ctx) ->
    match x state with
    | Value (y, s) -> f y s
    | Error e -> Error e
    
let (>>=) = bind

let (>>) f1 f2 = f1 >>= (fun () -> f2)

let (>>>) f1 f2 = f1 >>= (fun x -> f2 x)

let (let*) = bind

let get: ctx t = fun (ctx: ctx) -> Value(ctx, ctx)

let set: ctx -> unit t = fun (nctx: ctx) -> fun (ctx: ctx) -> Value ((), nctx)

type 'a t = ctx -> ('a * ctx) result


val return : 'a -> 'a t = <fun>


val error : errorinfo -> 'a t = <fun>


val bind : 'a t -> ('a -> 'b t) -> 'b t = <fun>


val ( >>= ) : 'a t -> ('a -> 'b t) -> 'b t = <fun>


val ( >> ) : unit t -> 'a t -> 'a t = <fun>


val ( >>> ) : 'a t -> ('a -> 'b t) -> 'b t = <fun>


val ( let* ) : 'a t -> ('a -> 'b t) -> 'b t = <fun>


val get : ctx t = <fun>


val set : ctx -> unit t = <fun>


In [16]:
let c = 
    let x1 = mkstate "x" Fresh TUnit in
    let x2 = mkstate "y" Used TUnit in
    [x1; x2]

val c : state list =
  [{var = "x"; used = Fresh; typ = TUnit};
   {var = "y"; used = Used; typ = TUnit}]


In [17]:
let rec empty (ctx: ctx) : bool =
    match ctx with
    | []                         -> true
    | x :: xs when x.used = Used -> empty xs
    | _                          -> false

val empty : ctx -> bool = <fun>


In [18]:
let rec lookup: var -> state t = fun (x: var) -> fun (ctx: ctx) ->
    match ctx with
    | []                     -> Error (fstring "Variable %s not in context" x)
    | y :: ys when x = y.var -> Value (y, y :: ys)
    | y :: ys                -> (lookup x >>= (fun s -> fun ctx' -> Value(s, y :: ctx'))) ys

let rec lookup_update: var -> state t = fun (x: var) -> fun (ctx: ctx) ->
    match ctx with
    | []                     -> Error (fstring "Variable %s not in context" x)
    | y :: ys when x = y.var -> Value (y, {y with used=Used} :: ys)
    | y :: ys                -> (lookup_update x >>= (fun s -> fun ctx' -> Value(s, y :: ctx'))) ys

val lookup : var -> state t = <fun>


val lookup_update : var -> state t = <fun>


In [19]:
let withvar: state -> 'a t -> 'a t = fun (x: state) -> fun (m: 'a t) ->
                                    let* ctx = get in
                                    let* r = ((set (x::ctx)) >> m) in
                                    let* s = lookup x.var in
                                    match s.used with
                                    | Used -> (set ctx) >> return r
                                    | Fresh -> error (fstring "Unused linear variable %s" x.var)

val withvar : state -> 'a t -> 'a t = <fun>


In [20]:
(*let rec withvars: state list -> 'a t -> 'a t = fun xs -> fun m ->
                                               match xs with
                                               | []    -> m
                                               | x::xs -> withvar x (withvars xs m)*)

In [21]:
let withvars: state list -> 'a t -> 'a t = fun xs -> fun m ->
                                           let* ctx = get in
                                           let* r = ((set (xs@ctx)) >> m) in
                                           let rec checker l =
                                               match l with
                                               | []    -> (set ctx) >> return ()
                                               | x::xs -> let* s = lookup x.var in
                                                          match s.used with
                                                          | Used -> checker xs
                                                          | Fresh -> error (fstring "Unused linear variable %s" s.var)
                                           in checker xs

val withvars : state list -> unit t -> unit t = <fun>


In [22]:
let c =
    let x1 = mkstate "x" Used TUnit in 
    let x2 = mkstate "y" Fresh TUnit in
[x1; x2];;
    
lookup_update "y" c

val c : state list =
  [{var = "x"; used = Used; typ = TUnit};
   {var = "y"; used = Fresh; typ = TUnit}]


- : (state * ctx) result =
Value
 ({var = "y"; used = Fresh; typ = TUnit},
  [{var = "x"; used = Used; typ = TUnit};
   {var = "y"; used = Used; typ = TUnit}])


In [23]:
let rec rm l v =
    match l with
    | x :: xs when v=x -> xs
    | x :: xs          -> x :: (rm xs v)
    | []               -> raise UnexpectedError

val rm : 'a list -> 'a -> 'a list = <fun>


In [24]:
let rec find l v = 
    match l with
    | x :: xs when v=x -> true
    | x :: xs          -> find xs v
    | []               -> false

val find : 'a list -> 'a -> bool = <fun>


In [25]:
let rec same (ctx1: ctx) (ctx2: ctx) : bool = 
    match ctx1, ctx2 with
    | s :: ss, _ -> if find ctx2 s then same ss (rm ctx2 s)
                                   else false
    | [], []     -> true
    | [], _      -> false

val same : ctx -> ctx -> bool = <fun>


In [26]:
let plsTensor: typ -> (typ*typ) t = fun t ->
    match t with
    | Tensor (t1, t2) -> return (t1, t2)
    | _ -> error (fstring "Expected something of the type α⊗β, got %s" (printtype t))

val plsTensor : typ -> (typ * typ) t = <fun>


In [27]:
let plsLoli: typ -> (typ*typ) t = fun t ->
    match t with
    | Loli (t1, t2) -> return (t1, t2)
    | _ -> error (fstring "Expected something of the type α⊸β, got %s" (printtype t))

val plsLoli : typ -> (typ * typ) t = <fun>


In [28]:
let plsLSum: typ -> (typ*typ) t = fun t ->
    match t with
    | Sum (t1, t2) -> return (t1, t2)
    | _ -> error (fstring "Expected something of the type α⊕β, got %s" (printtype t))

val plsLSum : typ -> (typ * typ) t = <fun>


In [29]:
check (Lambda("x", Annot(Var "x", TUnit))) (Loli(TUnit, TUnit)) []

error: compile_error

In [30]:
let rec check (e: expr) (t: typ) : unit t = 
    match e, t with
    | EUnit, TUnit -> return ()
    | Lambda (x, e'), Loli (t1, t2) -> withvar (fresh x t1) (check e' t2)
    | Pair (e1, e2), Tensor (t1, t2) -> check e1 t1 >> check e2 t2
    | Unpair (x1, x2, e1, e2), _ -> let* t1, t2 = infer e1 >>> plsTensor in
                                    withvars ([fresh x1 t1; fresh x2 t2]) (check e2 t2)
    | L(e'), Sum(t1, t2) -> check e' t1
    | R(e'), Sum(t1, t2) -> check e' t2
    | Case (e', x1, e1, x2, e2), _ -> let* t1, t2 = infer e' >>> plsLSum in
                                      let* ctx1 = withvar (fresh x1 t1) (check e1 t >> get) in
                                      let* ctx2 = withvar (fresh x2 t2) (check e2 t >> get) in
                                      if same ctx1 ctx2 then set ctx1 >> return () (*make same also monadic and make it error with a diff*)
                                                        else error (fstring "Different resulting contexts in case statement %s" (printexpr e))                                       
    | _, _ -> let* t' = infer e in
              if t=t' then return ()
                      else error (fstring "Can't type check %s with %s" (printexpr e) (printtype t))
and infer (e: expr) : typ t =
    match e with 
    | Var x -> let* s = lookup_update x in (
                   match s.used with
                   | Fresh -> return s.typ
                   | Used  -> error (fstring "Multiple usages of variable %s in linear context" x)
               )
    | Annot (e', t) -> let* () = check e' t in return t
    | App (e1, e2) -> let* t1, t2 = infer e1 >>> plsLoli in
                      check e1 t1 >> check e2 t2 >> return t2
    | _ -> error (fstring "Can't infer type of %s" (printexpr e))

val check : expr -> typ -> unit t = <fun>
val infer : expr -> typ t = <fun>


In [31]:
let typecheck (e: expr): typ t = 
    match e with
    | EUnit          -> (check e TUnit) >>= (fun (_: unit) -> return TUnit)
    | Var x          -> infer e
    | Annot (e', t)  -> infer e
    | App (e1, e2)   -> infer e
    | _ -> raise Unimplemented

val typecheck : expr -> typ t = <fun>


In [32]:
infer (Annot(Lambda("x", Var "x"), Loli(TUnit, TUnit))) []

- : (typ * ctx) result = Value (Loli (TUnit, TUnit), [])


In [33]:
check (Unpair("x1", "x2", Annot(Pair(Lambda("x", Var "x"), EUnit), Tensor(Loli(TUnit, TUnit), TUnit)),
                          App(Var "x1", Var "x2"))) TUnit []

- : (unit * ctx) result =
Error "Multiple usages of variable x1 in linear context"


In [34]:
typecheck (Annot(Lambda("x", Var "x"), Loli(TUnit, TUnit))) []

- : (typ * ctx) result = Value (Loli (TUnit, TUnit), [])


In [35]:
let checker = typecheck (App(Annot(Lambda("x", Var "x"), Loli(TUnit, TUnit)), Var "y"))

val checker : typ t = <fun>


In [36]:
checker [fresh "y" TUnit; fresh "z" TUnit]

- : (typ * ctx) result = Error "Can't type check (λx.x: (1⊸1)) with 1"


In [37]:
let c =
    let x1 = mkstate "x" Used TUnit in 
    let x2 = mkstate "y" Fresh TUnit in
[x1; x2];;

typecheck (Annot (Var "y", TUnit)) c;;

val c : state list =
  [{var = "x"; used = Used; typ = TUnit};
   {var = "y"; used = Fresh; typ = TUnit}]


- : (typ * ctx) result =
Value
 (TUnit,
  [{var = "x"; used = Used; typ = TUnit};
   {var = "y"; used = Used; typ = TUnit}])


In [38]:
empty c

- : bool = false
